In [90]:
%gui qt
import GPy
import numpy as np
from matplotlib import pyplot as plt
from pymodaq.utils.data import DataRaw, Axis
from pymodaq.utils import math_utils as mutils
from scipy.optimize import minimize
from scipy.stats import norm
import statistics
from copy import deepcopy

In [227]:
def laurentzian(x, x0, coeff, slope, alpha=1):
    return alpha * (coeff * slope) ** 2 / ((coeff * slope) ** 2 + (x - x0) ** 2)



def black_box(x: float):
    #return (np.sin(x) + np.cos(2*x) + np.cos(3*x)) * mutils.gauss1D(x, 0, np.pi) + 0.01 * np.random.random_sample(x.shape)
    return laurentzian(x, np.pi/6, 1, 0.1) + laurentzian(x, -1.5, 1, 0.25, 0.5) + laurentzian(x, 2, 1, 0.05, 1.1)  + 0.01 * np.random.random_sample(x.shape)

In [186]:
bounds = (-np.pi, np.pi)
axis = Axis('x', 'rad', data=np.linspace(bounds[0], bounds[1], 100, endpoint=True))
dwa = DataRaw('blackbox', data=[black_box(axis.get_data())], axes=[axis])
dwa_best = DataRaw('best', data=[np.array([0])], labels=['best'])
viewer_gp = dwa.plot('qt')
viewer_ucb = dwa.plot('qt')
viewer_best = dwa_best.plot('qt')

In [232]:
ini_points = 5

X = np.random.choice(axis.get_data(), (ini_points,))
X = np.expand_dims(X, 1)
Y = black_box(X)
ucb_x = []
dwa_choice = DataRaw('choice', data=[np.squeeze(black_box(X))], axes=[Axis('choices', data=np.squeeze(X))],
                     symbol_size=12,
                     symbol='d')

kernel = GPy.kern.Matern52(input_dim=1)
shape = (len(axis))

niter_max = 100
tol = 2e-2
kappa = 20
kappa_decay_rate = 0.98

stop_type = 'max_val'
stop_type = 'utility_predict'

for ind in range(niter_max):

    model_new = GPy.models.GPRegression(X, black_box(X), kernel)
    
    
    model_new.optimize()
    #m.plot(plot_raw=False)
    
    def ucb(x: np.ndarray, model):
        y_pred, y_std = model.predict(np.expand_dims(x, 1))
        #quantiles = m.predict_quantiles(np.expand_dims(x, 1))
        #ucb = y_pred + kappa * abs((y_pred - quantiles[0]))
        ucb = y_pred + kappa * np.sqrt(y_std)
        return ucb
    
    mean, variance = model_new.predict_noiseless(np.expand_dims(axis.get_data(), 1))
    quantiles = model_new.predict_quantiles(np.expand_dims(axis.get_data(), 1))
    #gradient =model_new.predictive_gradients(np.expand_dims(axis.get_data(), 1))
    
    ucb_plot = ucb(axis.get_data(), model_new)
    ucb_optimize = lambda x, model: -float(ucb(x, model)[0])
    ind_ucb_max = np.argmax(ucb_plot)
    x0 = axis.get_data_at(ind_ucb_max)
    dwa_predict = DataRaw('predict', data=[mean.reshape(shape),
                                           black_box(axis.get_data()).reshape(shape)],
                          axes=[axis],
                          errors=[np.sqrt(variance).reshape(shape) * 2,
                                  np.zeros(shape)
                                 ],
                         labels=['mean', 'black box'])
    
    dwa_choice = DataRaw('choice', data=[np.squeeze(black_box(X))],
                         axes=[Axis('choices', data=np.squeeze(X))],
                         labels=['Probed points'],
                         symbol_size=12,
                         symbol='d')
    viewer_gp = dwa_predict.plot('qt', scatter_dwa=dwa_choice, viewer=viewer_gp) 
    res = minimize(ucb_optimize, x0=x0, args=(model_new,), bounds=(bounds,),)
    ucb_x.append(res.x)
    dwa_ucb = DataRaw('ucb', data=[np.array([ucb_optimize(np.array([x]), model_new) for x in axis.get_data()])],
                      axes=[axis], labels=['ucb'])
    viewer_ucb = dwa_ucb.plot('qt', scatter_dwa=DataRaw('next', dim='Data1D',
                                                        data=[np.array([ucb_optimize(res.x, model_new)])],
                                           labels=['next'],
                                           axes=[Axis('next', data=res.x)]),
                             viewer=viewer_ucb)

    
    X = np.concatenate((X, np.expand_dims(res.x, 1)), axis=0)
    Y = np.concatenate((Y, np.expand_dims(black_box(res.x), 1)), axis=0)
    
    viewer_best.show_data(DataRaw('best', data=[black_box(res.x)], labels=['best']))

    #print(f'next probed point is: {res.x}')
    
    kappa *= kappa_decay_rate
    print(f'kappa is {kappa}')


    
    if ind > ini_points:

        
        if stop_type == 'max_val':
            stop_flag = (np.std(Y[-ini_points:,0]) < tol or ind >= niter_max)
        elif stop_type == 'utility_predict':
            stop_flag = (np.std(np.array(ucb_x)[-ini_points:,0]) < tol or ind >= niter_max)
        if stop_flag:
            break
            
print(f'optimisation done in {ind} steps. Best value reached at {X[np.argmax(Y)]}')


kappa is 19.6
kappa is 19.208000000000002
kappa is 18.82384
kappa is 18.4473632
kappa is 18.078415936000003
kappa is 17.716847617280003
kappa is 17.3625106649344
kappa is 17.01526045163571
kappa is 16.674955242602998
kappa is 16.34145613775094
kappa is 16.01462701499592
kappa is 15.694334474696001
kappa is 15.380447785202081
kappa is 15.072838829498039
kappa is 14.771382052908079
kappa is 14.475954411849917
kappa is 14.186435323612919
kappa is 13.90270661714066
kappa is 13.624652484797847
kappa is 13.35215943510189
kappa is 13.085116246399853
kappa is 12.823413921471856
kappa is 12.566945643042418
kappa is 12.31560673018157
kappa is 12.069294595577938
kappa is 11.82790870366638
kappa is 11.591350529593052
kappa is 11.359523519001192
kappa is 11.132333048621168
kappa is 10.909686387648746
kappa is 10.69149265989577
kappa is 10.477662806697854
kappa is 10.268109550563896
kappa is 10.062747359552617
kappa is 9.861492412361564
kappa is 9.664262564114333
kappa is 9.470977312832046
kappa is 